In [ ]:
import tensorflow as tf
import os 
import SimpleITK as sitk 
import json 
from tools.field_classification.instance import *
from tools.field_classification.inference import *  

In [ ]:
trained_model_path = '/media/oncopole/d508267f-cc7d-45e2-ae24-9456e005a01f/CLASSIFICATION/training/train_2/20210705-15_03_50/trained_model_20210705-15_03_50.h5'
model = tf.keras.models.load_model(trained_model_path)
model.summary()

In [ ]:
test_dataset_path = 'abs path json file of test dataset'
dataset = []
with open(test_dataset_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset.append(info)
X_test = []
y_test = []
for img_dict in dataset:
    mip, label = generate_instance_from_nifti(img_dict)
    X_test.append(mip)
    y_test.append(label)
    
X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

print(X_test.shape)
print(y_test.shape)

In [ ]:
#evaluate
result = model.evaluate(X_test,  {'head': y_test[:,0], 
                                    'legs': y_test[:,1],
                                    'right_arm' : y_test[:,2],
                                    'left_arm' : y_test[:,3] ,
                                    })
dict(zip(model.metrics_names, result)) 

In [ ]:
#predictions
inferences = model.predict(X_test)

In [ ]:
inferences_decoded = decodage_predictions(inferences)

In [ ]:
truth_decoded = decodage_truth(y_test)

In [ ]:
#generate image with true/false predictions and save them 
directory = 'directory where to save the images predictions'
generate_images_predictions(X_test, inferences_decoded, truth_decoded, directory)